# Ejemplo: Calificación con Umbrales

Este notebook demuestra cómo utilizar las funciones de calificación con umbrales del módulo `notas`.
Comparamos la versión simple (`calcula_promedio_con_umbrales_simple`) y la versión avanzada
(`calcula_promedio_con_umbrales_avanzado`).

## 0. Fundamentos Teóricos

A continuación se describen las fórmulas utilizadas para el cálculo de notas con penalización por interdependencia.

### 1) Nota ponderada normal (sin dependencias)
$$ A = \sum_{i=1}^n w_i x_i $$

### 2) Sistema de "pesos locales" (dependencia cruzada)
Para cada componente $i$, definimos su factor local como el producto de función umbral $g(\cdot)$ de **los otros**:
$$ U_i = \prod_{j \neq i} g(x_j) \quad \text{donde} \quad g(x) = \min\left(1, \frac{x}{\text{Umbral}}\right) $$

La nota "local penalizada" es:
$$ S_{local} = \sum_{i=1}^n w_i x_i U_i $$
*Interpretación: el aporte del componente $i$ se "desbloquea" solo si los demás componentes no están por debajo del umbral.*

### 3) Factor de Utilidad Único
Convertimos $S_{local}$ en un factor $F_{local} \in [0, 1]$:
$$ F_{local} = \begin{cases} \frac{S_{local}}{A}, & A > 0 \\ 1, & A = 0 \end{cases} $$

### 4) Nota Final Anclada (Piso Suave)
Para garantizar "no bajar del peor módulo", definimos un piso $m = \min_{i \in I} x_i$.
La nota final se interpola:
$$ N = m + (A - m) F_{local} $$
*Garantía: $N \ge m$*.

In [13]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import notas as nu
import config as cf

np.random.seed(42)
pd.set_option('display.float_format', '{:.2f}'.format)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Cargar Configuración y Generar Datos

In [14]:
# Configuración automática
config = nu.autoconfigura_items(cf.config_evaluacion)

# Generar datos de 20 estudiantes
df_notas = nu.genera_datos(config, N=20)
print("Datos generados:")
display(df_notas.head())

Autoconfigurando items correlacionados...
 > Quices_Mod1: Ajustado vs Tarea_Mod1 (x0.5)
 > Quices_Mod2: Ajustado vs Quices_Mod1 (x1.2)
 > Tarea_Mod2: Ajustado vs Tarea_Mod1 (x1.2)
 > Quices_Mod3: Ajustado vs Quices_Mod1 (x1.0)
 > Tarea_Mod3: Ajustado vs Tarea_Mod1 (x1.0)
Datos generados:


,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final
0,4.12,1.99,4.20,3.77,3.88,1.65,2.71
1,3.90,1.94,4.15,0.48,4.25,3.46,3.22
2,4.34,4.85,4.25,0.91,4.15,1.95,2.37
3,2.09,3.31,3.91,1.69,4.32,1.98,2.53
4,3.76,2.46,3.77,0.81,4.41,2.52,2.56


## 2. Calcular Notas (Método Simple vs Avanzado)

Calculamos las notas finales utilizando ambos métodos para comparar cómo aplican las penalizaciones.

In [15]:
# 1. Método Simple
notas_simple = nu.calcula_promedio_con_umbrales_simple(df_notas, config)

# 2. Método Avanzado (usa umbrales individuales automáticamente por defecto)
df_adv = nu.calcula_promedio_con_umbrales_avanzado(df_notas, config)
notas_avanzadas = df_adv['Nota_Final'].values

# 3. Promedio Clásico
promedio_clasico = df_adv['Promedio_Clasico'].values

# Crear DataFrame de comparación
df_res = df_notas.copy()
df_res['Promedio_Clasico'] = promedio_clasico
df_res['Nota_Simple'] = notas_simple
df_res['Nota_Avanzada'] = notas_avanzadas

cols_mostrar = ['Promedio_Clasico', 'Nota_Simple', 'Nota_Avanzada']
df_res[cols_mostrar].head(10)

,Promedio_Clasico,Nota_Simple,Nota_Avanzada
0,3.00,2.37,2.19
1,2.78,2.23,2.78
2,2.87,2.29,2.87
3,2.64,2.11,2.64
4,2.56,2.05,2.56
5,3.60,3.21,3.51
6,2.76,2.21,2.76
7,3.00,2.34,3.00
8,4.19,4.11,4.16
9,2.85,2.28,2.85


### Comparación de Estudiantes Penalizados
Identificamos estudiantes donde las notas calculadas difieren significativamente del promedio clásico.

In [16]:
df_res['Dif_Simple'] = df_res['Promedio_Clasico'] - df_res['Nota_Simple']
df_res['Dif_Avanzada'] = df_res['Promedio_Clasico'] - df_res['Nota_Avanzada']

penalizados = df_res[(df_res['Dif_Simple'] > 0.01) | (df_res['Dif_Avanzada'] > 0.01)]
print(f"Estudiantes penalizados por alguno de los métodos: {len(penalizados)} de {len(df_res)}")

cols_pen = ['Promedio_Clasico', 'Nota_Simple', 'Nota_Avanzada', 'Dif_Simple', 'Dif_Avanzada']
penalizados[cols_pen].sort_values('Dif_Avanzada', ascending=False).head(10)

Estudiantes penalizados por alguno de los métodos: 20 de 20


,Promedio_Clasico,Nota_Simple,Nota_Avanzada,Dif_Simple,Dif_Avanzada
17,3.13,0.68,1.39,2.45,1.74
19,3.25,1.17,2.08,2.08,1.17
18,3.37,1.48,2.27,1.89,1.10
16,3.29,1.39,2.30,1.90,0.99
0,3.00,2.37,2.19,0.63,0.81
13,3.33,2.09,2.75,1.24,0.58
10,3.00,2.37,2.52,0.63,0.48
11,3.60,2.61,3.18,0.99,0.42
5,3.60,3.21,3.51,0.39,0.09
8,4.19,4.11,4.16,0.08,0.03


## 3. Ejemplo Detallado: Estudiante 11

Este estudiante representa un caso interesante donde **se aprueba con el promedio clásico** pero **se reprueba con el método simple** debido a la penalización estricta.

Usamos la función `mostrar_detalle_estudiante` para ver el desglose paso a paso.

In [17]:
estudiante_id = 11
nu.mostrar_detalle_estudiante(estudiante_id, df_notas, config)


ESTUDIANTE ID: 11

1. NOTAS Y PESOS:
   Quices_Mod1     :  3.45 (Peso: 7.50%)
   Tarea_Mod1      :  2.09 (Peso: 15.00%)
   Quices_Mod2     :  3.87 (Peso: 9.00%)
   Tarea_Mod2      :  4.56 (Peso: 18.00%)
   Quices_Mod3     :  2.21 (Peso: 7.50%)
   Tarea_Mod3      :  3.13 (Peso: 15.00%)
   Examen Final    :  4.38 (Peso: 28.00%)

   PROMEDIO CLÁSICO : 3.6030

==================== MÉTODO SIMPLE ====================

2. FACTORES DE UMBRAL (Simple Strict):
   Ratio Quices_Mod1  (Umbral 2.2): 1.00
   Ratio Tarea_Mod1   (Umbral 3.0): 0.70
   Ratio Quices_Mod2  (Umbral 2.5): 1.00
   Ratio Tarea_Mod2   (Umbral 2.8): 1.00
   Ratio Quices_Mod3  (Umbral 2.0): 1.00
   Ratio Tarea_Mod3   (Umbral 3.0): 1.00
   Ratio Examen Final (Umbral 2.5): 1.00

3. FACTORES DE UTILIDAD (U_i):
   U_Quices_Mod1  = 0.6967  [= Tare(0.70) * Quic(1.00) * Tare(1.00) * Quic(1.00) * Tare(1.00) * Exam(1.00)]
   U_Tarea_Mod1   = 1.0000  [= Quic(1.00) * Quic(1.00) * Tare(1.00) * Quic(1.00) * Tare(1.00) * Exam(1.00)]
   U_Quic

In [18]:
estudiante_id = 19
nu.mostrar_detalle_estudiante(estudiante_id, df_notas, config)


ESTUDIANTE ID: 19

1. NOTAS Y PESOS:
   Quices_Mod1     :  4.45 (Peso: 7.50%)
   Tarea_Mod1      :  3.07 (Peso: 15.00%)
   Quices_Mod2     :  2.26 (Peso: 9.00%)
   Tarea_Mod2      :  1.42 (Peso: 18.00%)
   Quices_Mod3     :  3.72 (Peso: 7.50%)
   Tarea_Mod3      :  2.09 (Peso: 15.00%)
   Examen Final    :  5.00 (Peso: 28.00%)

   PROMEDIO CLÁSICO : 3.2458

==================== MÉTODO SIMPLE ====================

2. FACTORES DE UMBRAL (Simple Strict):
   Ratio Quices_Mod1  (Umbral 2.2): 1.00
   Ratio Tarea_Mod1   (Umbral 3.0): 1.00
   Ratio Quices_Mod2  (Umbral 2.5): 0.90
   Ratio Tarea_Mod2   (Umbral 2.8): 0.51
   Ratio Quices_Mod3  (Umbral 2.0): 1.00
   Ratio Tarea_Mod3   (Umbral 3.0): 0.70
   Ratio Examen Final (Umbral 2.5): 1.00

3. FACTORES DE UTILIDAD (U_i):
   U_Quices_Mod1  = 0.3194  [= Tare(1.00) * Quic(0.90) * Tare(0.51) * Quic(1.00) * Tare(0.70) * Exam(1.00)]
   U_Tarea_Mod1   = 0.3194  [= Quic(1.00) * Quic(0.90) * Tare(0.51) * Quic(1.00) * Tare(0.70) * Exam(1.00)]
   U_Quic